In [1]:
import os
import json
import gc
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer
from typing import Dict, List, Set, Tuple, NamedTuple, Callable
import scipy
import scml
from scml import pandasx as pdx
from daigt.preprocess.en import BasicPreprocessor, BowPreprocessor
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.iinfo(np.int16)
print(f"int16, min={info.min}, max={info.max}")

int16, min=-32768, max=32767


In [2]:
tokenizer = AutoTokenizer.from_pretrained("huggingface/microsoft/deberta-v3-base", is_fast=True)
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")

DebertaV2TokenizerFast(name_or_path='huggingface/microsoft/deberta-v3-base', vocab_size=128000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '[CLS]', 'eos_token': '[SEP]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	128000: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', '

/Users/ruhong/github/seahrh/kaggle-detect-ai-generated-text-2023/venv/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [3]:
edf = pd.read_csv("input/train_essays.csv", low_memory=False)
cols = ["generated"]
edf[cols] = edf[cols].astype(np.int8)
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1378 non-null   object
 1   prompt_id  1378 non-null   int64 
 2   text       1378 non-null   object
 3   generated  1378 non-null   int8  
dtypes: int64(1), int8(1), object(2)
memory usage: 33.8+ KB


In [4]:
cols=["prompt_id", "generated"]
pdx.value_counts(edf[cols])

count   percent
prompt_id generated                 
0         0            707  0.513062
1         0            668  0.484761
          1              2  0.001451
0         1              1  0.000726

In [5]:
pdf = pd.read_csv("input/train_prompts.csv", low_memory=False)
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   prompt_id     2 non-null      int64 
 1   prompt_name   2 non-null      object
 2   instructions  2 non-null      object
 3   source_text   2 non-null      object
dtypes: int64(1), object(3)
memory usage: 196.0+ bytes


# Preprocess Text

In [6]:
bsc = BasicPreprocessor()
bow = BowPreprocessor()

def preprocess_bsc(col) -> Callable:
    def fn(row) -> str:
        return bsc(row[col])
    
    return fn

def preprocess_bow(col) -> Callable:
    def fn(row) -> str:
        return bow(row[col], drop_stopword=False)
    
    return fn

In [7]:
dfs = [pdf, edf]
cols = [["prompt_name", "instructions", "source_text"],["text"]]
for i, df in enumerate(dfs):
    for col in cols[i]:
        print(col)
        new_col = f"{col}_bsc" 
        df[new_col] = df.progress_apply(preprocess_bsc(col), axis=1)
        new_col = f"{col}_bow" 
        df[new_col] = df.progress_apply(preprocess_bow(col), axis=1)
        x = tokenizer(df[new_col].tolist(), truncation=False, add_special_tokens=False)
        len_col = f"{new_col}_len" 
        df[len_col] = [len(s) for s in x["input_ids"]]
        df[len_col] = df[len_col].astype(np.int16) 

prompt_name


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 512.94it/s]


instructions


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 132.35it/s]


source_text


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  6.73it/s]


text


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1378/1378 [00:50<00:00, 27.38it/s]


In [8]:
#more = len(df)
#df = df.drop_duplicates(["prompt_id", "text_bow"], ignore_index=True)
#print(f"{more - len(df)} rows dropped: duplicates")

# Review Data

In [9]:
edf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   id            1378 non-null   object
 1   prompt_id     1378 non-null   int64 
 2   text          1378 non-null   object
 3   generated     1378 non-null   int8  
 4   text_bsc      1378 non-null   object
 5   text_bow      1378 non-null   object
 6   text_bow_len  1378 non-null   int16 
dtypes: int16(1), int64(1), int8(1), object(4)
memory usage: 58.0+ KB


In [10]:
edf.describe(percentiles=percentiles)

,prompt_id,generated,text_bow_len
count,1378.000000,1378.000000,1378.000000
mean,0.486212,0.002177,585.858491
std,0.499991,0.046625,180.131658
min,0.000000,0.000000,237.000000
1%,0.000000,0.000000,311.770000
5%,0.000000,0.000000,369.850000
10%,0.000000,0.000000,401.000000
20%,0.000000,0.000000,448.000000
30%,0.000000,0.000000,484.100000
40%,0.000000,0.000000,516.000000


In [11]:
edf.head(2)

,id,prompt_id,text,generated,text_bsc,text_bow,text_bow_len
0,0059830c,0,"Cars. Cars have been around since they became famous in the 1900s, when Henry Ford created and built the first ModelT. Cars have played a major role in our every day lives since then. But now, people are starting to question if limiting car usage would be a good thing. To me, limiting the use of cars might be a good thing to do.\n\nIn like matter of this, article, ""In German Suburb, Life Goes On Without Cars,"" by Elizabeth Rosenthal states, how automobiles are the linchpin of suburbs, where middle class families from either Shanghai or Chicago tend to make their homes. Experts say how this is a huge impediment to current efforts to reduce greenhouse gas emissions from tailpipe. Passenger cars are responsible for 12 percent of greenhouse gas emissions in Europe...and up to 50 percent in some carintensive areas in the United States. Cars are the main reason for the greenhouse gas emissions because of a lot of people driving them around all the time getting where they need to go. Article, ""Paris bans driving due to smog,"" by Robert Duffer says, how Paris, after days of nearrecord pollution, enforced a partial driving ban to clear the air of the global city. It also says, how on Monday, motorist with evennumbered license plates were ordered to leave their cars at home or be fined a 22euro fine 31. The same order would be applied to oddnumbered plates the following day. Cars are the reason for polluting entire cities like Paris. This shows how bad cars can be because, of all the pollution that they can cause to an entire city.\n\nLikewise, in the article, ""Carfree day is spinning into a big hit in Bogota,"" by Andrew Selsky says, how programs that's set to spread to other countries, millions of Columbians hiked, biked, skated, or took the bus to work during a carfree day, leaving streets of this capital city eerily devoid of traffic jams. It was the third straight year cars have been banned with only buses and taxis permitted for the Day Without Cars in the capital city of 7 million. People like the idea of having carfree days because, it allows them to lesson the pollution that cars put out of their exhaust from people driving all the time. The article also tells how parks and sports centers have bustled throughout the city uneven, pitted sidewalks have been replaced by broad, smooth sidewalks rushhour restrictions have dramatically cut traffic and new restaurants and upscale shopping districts have cropped up. Having no cars has been good for the country of Columbia because, it has aloud them to repair things that have needed repairs for a long time, traffic jams have gone down, and restaurants and shopping districts have popped up, all due to the fact of having less cars around.\n\nIn conclusion, the use of less cars and having carfree days, have had a big impact on the environment of cities because, it is cutting down the air pollution that the cars have majorly polluted, it has aloud countries like Columbia to repair sidewalks, and cut down traffic jams. Limiting the use of cars would be a good thing for America. So we should limit the use of cars by maybe riding a bike, or maybe walking somewhere that isn't that far from you and doesn't need the use of a car to get you there. To me, limiting the use of cars might be a good thing to do.",0,"Cars. Cars have been around since they became famous in the 1900s, when Henry Ford created and built the first ModelT. Cars have played a major role in our every day lives since then. But now, people are starting to question if limiting car usage would be a good thing. To me, limiting the use of cars might be a good thing to do. In like matter of this, article, ""In German Suburb, Life Goes On Without Cars,"" by Elizabeth Rosenthal states, how automobiles are the linchpin of suburbs, where middle class families from either Shanghai or Chicago tend to make their homes. Experts say how this is a huge impediment to 

In [12]:
pdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   prompt_id             2 non-null      int64 
 1   prompt_name           2 non-null      object
 2   instructions          2 non-null      object
 3   source_text           2 non-null      object
 4   prompt_name_bsc       2 non-null      object
 5   prompt_name_bow       2 non-null      object
 6   prompt_name_bow_len   2 non-null      int16 
 7   instructions_bsc      2 non-null      object
 8   instructions_bow      2 non-null      object
 9   instructions_bow_len  2 non-null      int16 
 10  source_text_bsc       2 non-null      object
 11  source_text_bow       2 non-null      object
 12  source_text_bow_len   2 non-null      int16 
dtypes: int16(3), int64(1), object(9)
memory usage: 304.0+ bytes


In [13]:
pdf.describe(percentiles=percentiles)

,prompt_id,prompt_name_bow_len,instructions_bow_len,source_text_bow_len
count,2.000000,2.000000,2.000000,2.000000
mean,0.500000,4.000000,96.500000,2195.500000
std,0.707107,1.414214,10.606602,135.057395
min,0.000000,3.000000,89.000000,2100.000000
1%,0.010000,3.020000,89.150000,2101.910000
5%,0.050000,3.100000,89.750000,2109.550000
10%,0.100000,3.200000,90.500000,2119.100000
20%,0.200000,3.400000,92.000000,2138.200000
30%,0.300000,3.600000,93.500000,2157.300000
40%,0.400000,3.800000,95.000000,2176.400000


In [14]:
pdf.head(2)

prompt_id                       prompt_name  \
0          0                   Car-free cities   
1          1  Does the electoral college work?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             instructions  \
0                                                                                        Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Your essay must be based on ideas and information that can be found in the passage set. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your essay in the space provided.   
1  Write a letter to your state senator in which you argue in favor of keeping the Electoral College or changing to election by popular vote for the president of the United States. Use the information from the texts in your essay. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to include a claim; address counterclaims; use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your response in the space provided.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [15]:
%%time
edf.to_parquet(f"output/essays_tra.parquet", index=False)
pdf.to_parquet(f"output/prompts_tra.parquet", index=False)
assert edf.notna().all(axis=None)
assert pdf.notna().all(axis=None)

CPU times: user 42.1 ms, sys: 7.03 ms, total: 49.1 ms
Wall time: 48.8 ms


In [16]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:00:53.239136
